In [17]:
import csv
import json
import time
import re
from datetime import datetime
from pathlib import Path

import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
MODEL_NAME = "gpt-4o-mini"
TEMPERATURE = 0.9
TOP_P = 0.95
MAX_TOKENS = 2000

NUM_PAIRS_PER_COMBO = 20     # 한 조합당 생성할 문장 쌍 수
MAX_RETRIES = 3              # LLM 호출 재시도 횟수
SLEEP_BETWEEN_CALLS = 0.2    # 호출 사이 간격(초)
PROMPT_VERSION = "v2_structured"

OUTPUT_DIR = Path("data")
OUTPUT_DIR.mkdir(exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

csv_path = OUTPUT_DIR / f"parallel_data_{MODEL_NAME}_{PROMPT_VERSION}_{timestamp}.csv"

In [ ]:
# 1. LLM 인스턴스 생성
llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    max_tokens=MAX_TOKENS,
)

# 2. 주제 / 스타일 / 문장 타입 목록
topics = [
    "여행", "일상생활", "건강", "식당", "기술", "업무", "교육", "시간관리",
    "감정표현", "경제/금융", "스포츠", "기후/날씨", "교통", "고객센터",
    "문화/예술", "쇼핑", "직장 관계", "IT/AI", "자기계발", "사회/뉴스",
]

styles = [
    "대화체",
    "문어체(보고서 스타일)",
    "SNS 스타일",
    "뉴스 기사체",
    "기술 설명체",
    "감정 표현 스타일",
    "조언/상담 스타일",
    "고객센터 안내 스타일",
]

types = [
    "단문",
    "복문",
    "조건문",
    "비교문",
    "원인·결과문",
    "명령/요청문",
]

# 3. JSON 구조화 출력 프롬프트
prompt_template = ChatPromptTemplate.from_template(
    """
너는 한국어-영어 번역 데이터셋을 만드는 어시스턴트야.

주제: {topic}
문체 스타일: {style}
문장 구조 타입: {type}
생성할 문장 쌍 수: {num_pairs}

조건:
- 반드시 한국어-영어 병렬 문장 {num_pairs}쌍을 생성할 것
- 각 문장은 서로 다른 구조와 길이, 표현을 사용할 것
- 한국어와 영어는 의미적으로 정확히 대응할 것
- 직역투를 피하고 자연스러운 표현을 사용할 것
- 한국어 문장 길이는 대략 10~40자, 영어 문장 길이는 대략 10~80자 정도로 할 것
- 중복 표현을 피할 것
- 반말, 존댓말(해요체), 격식체(합니다체)를 전체 데이터셋에서 골고루 섞어서 사용하되,
  각 문장 안에서는 말투를 일관되게 유지할 것

출력 형식(중요):
- 반드시 아래 JSON 형식의 "하나의 객체"만 출력해
- JSON 이외의 설명, 마크다운, 코드블록, 주석, 텍스트를 절대 추가하지 마

출력 예시 형식(구조만 참고):

- "pairs" 키 아래에 한국어-영어 문장 쌍 리스트를 둔다고 생각하고 작성해.
규칙:
- key 이름은 반드시 "pairs", "ko", "en"만 사용해
- "pairs"는 길이가 {num_pairs}인 리스트여야 해
- JSON 문법 오류가 없도록 유효한 JSON만 출력해
    """
)

# 4. LLM 출력 파싱 함수 (한 호출 결과 → (ko, en) 리스트)
def parse_output(response):
    text = getattr(response, "content", str(response))
    try:
        data = json.loads(text)
    except json.JSONDecodeError as e:
        print("[JSON 파싱 실패]", e)
        return []

    raw_pairs = data.get("pairs", [])
    cleaned_pairs = []
    for item in raw_pairs:
        ko = str(item.get("ko", "")).strip()
        en = str(item.get("en", "")).strip()
        if not ko or not en:
            continue

        # 길이 필터링
        if not (10 <= len(ko) <= 80):
            continue
        if not (10 <= len(en) <= 120):
            continue

        cleaned_pairs.append((ko, en))

    return cleaned_pairs

# 5. 체인 정의 (프롬프트 → LLM → 파싱)
chain = prompt_template | llm | RunnableLambda(parse_output)

# 6. 전체 조합 입력 만들기
combos = [
    {"topic": t, "style": s, "type": ty, "num_pairs": NUM_PAIRS_PER_COMBO}
    for t in topics
    for s in styles
    for ty in types
]
total_combos = len(combos)
print(f"총 조합 수: {total_combos} (topic x style x type)")

# 7. 병렬(batch) 호출
#    max_concurrency 값을 4~16 사이에서 조절해 보세요.
MAX_CONCURRENCY = 8
results = chain.batch(combos, {"max_concurrency": MAX_CONCURRENCY})

# 8. 결과 모으기
rows = []
seen_pairs = set()

for idx, (combo, pairs) in enumerate(zip(combos, results), start=1):
    topic = combo["topic"]
    style = combo["style"]
    type_ = combo["type"]

    unique_pairs = []
    for ko, en in pairs:
        key = (ko, en)
        if key in seen_pairs:
            continue
        seen_pairs.add(key)
        unique_pairs.append((ko, en))

    for ko, en in unique_pairs:
        rows.append(
            {
                "id": len(rows) + 1,
                "topic": topic,
                "style": style,
                "type": type_,
                "ko": ko,
                "en": en,
                "model": MODEL_NAME,
                "prompt_version": PROMPT_VERSION,
                "created_at": timestamp,
            }
        )

    print(
        f"[생성 완료] [{idx}/{total_combos}] {topic} / {style} / {type_} → "
        f"{len(unique_pairs)}쌍 추가 (누적 {len(rows)}쌍)"
    )

# 9. CSV 저장 + train/valid/test 분할
if not rows:
    print("[경고] 생성된 데이터가 없습니다.")
else:
    df = pd.DataFrame(rows)
    df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)

    train_ratio = 0.8
    valid_ratio = 0.1  # 나머지 0.1은 test

    n = len(df)
    train_end = int(n * train_ratio)
    valid_end = int(n * (train_ratio + valid_ratio))

    df["split"] = "train"
    df.loc[train_end:valid_end, "split"] = "valid"
    df.loc[valid_end:, "split"] = "test"

    df.to_csv(csv_path, index=False, encoding="utf-8")

    train_path = csv_path.with_name("train.csv")
    valid_path = csv_path.with_name("valid.csv")
    test_path = csv_path.with_name("test.csv")

    df[df["split"] == "train"].to_csv(train_path, index=False, encoding="utf-8")
    df[df["split"] == "valid"].to_csv(valid_path, index=False, encoding="utf-8")
    df[df["split"] == "test"].to_csv(test_path, index=False, encoding="utf-8")

    print(f"\n[저장 완료] 전체: {csv_path}")
    print(f"[저장 완료] train: {train_path}")
    print(f"[저장 완료] valid: {valid_path}")
    print(f"[저장 완료] test:  {test_path}")

총 조합 수: 960 (topic x style x type)
[생성 완료] [1/960] 여행 / 대화체 / 단문 → 9쌍 추가 (누적 9쌍)
[생성 완료] [2/960] 여행 / 대화체 / 복문 → 10쌍 추가 (누적 19쌍)
[생성 완료] [3/960] 여행 / 대화체 / 조건문 → 10쌍 추가 (누적 29쌍)
[생성 완료] [4/960] 여행 / 대화체 / 비교문 → 10쌍 추가 (누적 39쌍)
[생성 완료] [5/960] 여행 / 대화체 / 원인·결과문 → 10쌍 추가 (누적 49쌍)
[생성 완료] [6/960] 여행 / 대화체 / 명령/요청문 → 10쌍 추가 (누적 59쌍)
[생성 완료] [7/960] 여행 / 문어체(보고서 스타일) / 단문 → 10쌍 추가 (누적 69쌍)
[생성 완료] [8/960] 여행 / 문어체(보고서 스타일) / 복문 → 10쌍 추가 (누적 79쌍)
[생성 완료] [9/960] 여행 / 문어체(보고서 스타일) / 조건문 → 10쌍 추가 (누적 89쌍)
[생성 완료] [10/960] 여행 / 문어체(보고서 스타일) / 비교문 → 10쌍 추가 (누적 99쌍)
[생성 완료] [11/960] 여행 / 문어체(보고서 스타일) / 원인·결과문 → 10쌍 추가 (누적 109쌍)
[생성 완료] [12/960] 여행 / 문어체(보고서 스타일) / 명령/요청문 → 10쌍 추가 (누적 119쌍)
[생성 완료] [13/960] 여행 / SNS 스타일 / 단문 → 10쌍 추가 (누적 129쌍)
[생성 완료] [14/960] 여행 / SNS 스타일 / 복문 → 10쌍 추가 (누적 139쌍)
[생성 완료] [15/960] 여행 / SNS 스타일 / 조건문 → 10쌍 추가 (누적 149쌍)
[생성 완료] [16/960] 여행 / SNS 스타일 / 비교문 → 10쌍 추가 (누적 159쌍)
[생성 완료] [17/960] 여행 / SNS 스타일 / 원인·결과문 → 10쌍 추가 (누적 169쌍)
[생성 완료] [18/960] 여행 / SNS 스타일 / 명령/요